In [1]:
import os, glob,pickle,sys,copy

import numpy as np
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
myfontsize=22
plt.rcParams.update({'font.size': myfontsize})
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,ScalarFormatter, NullFormatter,MaxNLocator, NullLocator,LogLocator


from astropy.table import Table
import astropy.io.fits as pyfits
import astropy.units as u
#from scipy.stats import gaussian_kde
#from scipy.stats import mode

from grizli import utils,prep,multifit,fitting
from grizli.pipeline import auto_script,photoz
import eazy
import warnings
warnings.filterwarnings('ignore')
root = 'uma-03'
prep_dir= '/Users/lkawinwanichakij/NISPureParallel/FIELDS/%s/Prep'  % root # modify if needed
extract_dir= '/Users/lkawinwanichakij/NISPureParallel/FIELDS/%s/Extractions'  % root
"""
Load neccessary files for run Grizli
"""
fit_args_file=extract_dir+'/fit_args_wNIRISSphot.npy'
with open(extract_dir+'/outthere-niriss-%s_eazyphot.pkl' % root, "rb") as f:
     int_phot_obj = pickle.load(f)
args=  np.load(fit_args_file, allow_pickle=True)[0]

In [2]:
os.getcwd()

'/Users/lkawinwanichakij/JWST/NIRISS/OutThere/uma-03/Grizli_redshiftfit_NIRISS_NIRCam_3dhst_photometry'

In [3]:
phot_cat= Table.read('uma03-nircam+niriss+3dhst_photcat_4eazy.fits')
# if 'z_spec' not in phot_cat.colnames:
#     phot_cat['z_spec'] = np.ones(len(phot_cat))*(-1)
#     phot_cat.write('uma-03_nircam+niriss+3dhst_photcat_4eazy.fits',overwrite=True)
phot_cat.colnames

['id',
 'ra',
 'dec',
 'f_f150wn_niriss',
 'e_f150wn_niriss',
 'f_f200wn_niriss',
 'e_f200wn_niriss',
 'f_f160w_3dhst',
 'e_f160w_3dhst',
 'f_u_3dhst',
 'e_u_3dhst',
 'f_f435w_3dhst',
 'e_f435w_3dhst',
 'f_b_3dhst',
 'e_b_3dhst',
 'f_g_3dhst',
 'e_g_3dhst',
 'f_v_3dhst',
 'e_v_3dhst',
 'f_f606w_3dhst',
 'e_f606w_3dhst',
 'f_r_3dhst',
 'e_r_3dhst',
 'f_rs_3dhst',
 'e_rs_3dhst',
 'f_i_3dhst',
 'e_i_3dhst',
 'f_f775w_3dhst',
 'e_f775w_3dhst',
 'f_z_3dhst',
 'e_z_3dhst',
 'f_f850lp_3dhst',
 'e_f850lp_3dhst',
 'f_f125w_3dhst',
 'e_f125w_3dhst',
 'f_j_3dhst',
 'e_j_3dhst',
 'f_f140w_3dhst',
 'e_f140w_3dhst',
 'f_h_3dhst',
 'e_h_3dhst',
 'f_ks_3dhst',
 'e_ks_3dhst',
 'f_irac1_3dhst',
 'e_irac1_3dhst',
 'f_irac2_3dhst',
 'e_irac2_3dhst',
 'f_irac3_3dhst',
 'e_irac3_3dhst',
 'f_irac4_3dhst',
 'e_irac4_3dhst',
 'f_f444w-f405n_nircam',
 'e_f444w-f405n_nircam',
 'f_f410m_nircam',
 'e_f410m_nircam',
 'f_f182m_nircam',
 'e_f182m_nircam',
 'f_f444w_nircam',
 'e_f444w_nircam',
 'f_f187n_nircam',
 'e_f

In [4]:
### Initialize **eazy.photoz** object for JADES

params={}
params['CATALOG_FILE'] = 'uma03-nircam+niriss+3dhst_photcat_4eazy.fits'
params['Z_STEP'] = 0.002
params['Z_MAX'] = 12
params['MAIN_OUTPUT_FILE'] = 'uma-03-nircam+niriss+3dhst_phot.eazypy'
params['PRIOR_FILTER'] = 205 # f160w
# Galactic extinction
#params['MW_EBV'] = {'aegis':0.0066, 'cosmos':0.0148, 'goodss':0.0069, 
#                    'uds':0.0195, 'goodsn':0.0103}[field]
params['MW_EBV'] = 0.0103
params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
params['PRIOR_FILE']='templates/prior_F160W_TAO.dat'
translate_file = 'nircam+niriss+3dhst.translate'
jades_ez = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, 
                        zeropoint_file=None, params=params, 
                        load_prior=True, load_products=False)
## First-pass redshift templates, similar to the eazy templates but 
## with separate emission lines
t0 = args['t0']
## Make a helper object for generating photometry in a format that grizli 
## understands.
phot_obj = photoz.EazyPhot(jades_ez , grizli_templates=t0, zgrid=jades_ez.zgrid)
with open('uma-03-nircam+niriss+3dhst_eazyphot.pkl', "wb") as file:  # Open the file in binary write mode
    pickle.dump(phot_obj, file)

Read default param file: /opt/anaconda3/envs/jadc/lib/python3.12/site-packages/eazy/data/zphot.param.default
Read CATALOG_FILE: uma03-nircam+niriss+3dhst_photcat_4eazy.fits
   >>> NOBJ = 20337
f_f160w_3dhst e_f160w_3dhst (205): hst/wfc3/IR/f160w.dat
f_u_3dhst e_u_3dhst (227): NOAO/k1001bp_jul04.txt
f_f435w_3dhst e_f435w_3dhst (233): hst/ACS_update_sep07/wfc_f435w_t81.dat
f_g_3dhst e_g_3dhst (229): LRIS/g_blue_transmission.dat
f_f606w_3dhst e_f606w_3dhst (236): hst/ACS_update_sep07/wfc_f606w_t81.dat
f_rs_3dhst e_rs_3dhst (231): LRIS/Rs_LRISred_transmission.dat
f_f775w_3dhst e_f775w_3dhst (238): hst/ACS_update_sep07/wfc_f775w_t81.dat
f_f850lp_3dhst e_f850lp_3dhst (240): hst/ACS_update_sep07/wfc_f850lp_t81.dat
f_f125w_3dhst e_f125w_3dhst (203): hst/wfc3/IR/f125w.dat
f_j_3dhst e_j_3dhst (224): MOIRCS/J277.txt
f_f140w_3dhst e_f140w_3dhst (204): hst/wfc3/IR/f140w.dat
f_h_3dhst e_h_3dhst (225): MOIRCS/H117.txt
f_ks_3dhst e_ks_3dhst (226): MOIRCS/Ks_rot1707wedged120K.txt
f_irac1_3dhst e_irac1_

100%|███████████████████████████████████████████| 12/12 [00:07<00:00,  1.55it/s]


Template   0: tweak_fsps_QSF_12_v3_001.dat (NZ=1).
Template   1: tweak_fsps_QSF_12_v3_002.dat (NZ=1).
Template   2: tweak_fsps_QSF_12_v3_003.dat (NZ=1).
Template   3: tweak_fsps_QSF_12_v3_004.dat (NZ=1).
Template   4: tweak_fsps_QSF_12_v3_005.dat (NZ=1).
Template   5: tweak_fsps_QSF_12_v3_006.dat (NZ=1).
Template   6: tweak_fsps_QSF_12_v3_007.dat (NZ=1).
Template   7: tweak_fsps_QSF_12_v3_008.dat (NZ=1).
Template   8: tweak_fsps_QSF_12_v3_009.dat (NZ=1).
Template   9: tweak_fsps_QSF_12_v3_010.dat (NZ=1).
Template  10: tweak_fsps_QSF_12_v3_011.dat (NZ=1).
Template  11: tweak_fsps_QSF_12_v3_012.dat (NZ=1).
Process templates: 7.982 s


1278it [00:05, 238.02it/s]
100%|███████████████████████████████████████████| 17/17 [00:24<00:00,  1.44s/it]

Template   0: fsps/fsps_QSF_12_v3_nolines_001.dat (NZ=1).
Template   1: fsps/fsps_QSF_12_v3_nolines_002.dat (NZ=1).
Template   2: fsps/fsps_QSF_12_v3_nolines_003.dat (NZ=1).
Template   3: fsps/fsps_QSF_12_v3_nolines_004.dat (NZ=1).
Template   4: fsps/fsps_QSF_12_v3_nolines_005.dat (NZ=1).
Template   5: fsps/fsps_QSF_12_v3_nolines_006.dat (NZ=1).
Template   6: fsps/fsps_QSF_12_v3_nolines_007.dat (NZ=1).
Template   7: fsps/fsps_QSF_12_v3_nolines_008.dat (NZ=1).
Template   8: fsps/fsps_QSF_12_v3_nolines_009.dat (NZ=1).
Template   9: fsps/fsps_QSF_12_v3_nolines_010.dat (NZ=1).
Template  10: fsps/fsps_QSF_12_v3_nolines_011.dat (NZ=1).
Template  11: fsps/fsps_QSF_12_v3_nolines_012.dat (NZ=1).
Template  12: alf_SSP.dat (NZ=1).
Template  13: line Ha+NII+SII+SIII+He+PaB (NZ=1).
Template  14: line OIII+Hb+Hg+Hd (NZ=1).
Template  15: line OII+Ne (NZ=1).
Template  16: line Gal-UV-lines (NZ=1).
